<a href="https://colab.research.google.com/github/ouhoucht-mohammed/mini_projet_NLP/blob/main/Mini_Projet_NLP_Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing libraries**

In [ ]:
#library Pytorch

!pip install torch===1.5.0 torchvision===0.6.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [ ]:
#library transformers

!pip install transformers==2.9.1

In [ ]:
# the web dashboard framework

!pip install panel

**Loading the Model**

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from random import choice

# import the OpenGPT2 Tokenizer and the Language Model

tok = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

**Predictions Function**

In [ ]:
def get_pred(text, model, tok, p=0.7):
    input_ids = torch.tensor(tok.encode(text)).unsqueeze(0)
    logits = model(input_ids)[0][:, -1]
    probs = F.softmax(logits, dim=-1).squeeze()
    idxs = torch.argsort(probs, descending=True)
    res, cumsum = [], 0.
    for idx in idxs:
        res.append(idx)
        cumsum += probs[idx]
        if cumsum > p:
            pred_idx = idxs.new_tensor([choice(res)])
            break
    pred = tok.convert_ids_to_tokens(int(pred_idx))
    return tok.convert_tokens_to_string(pred)

**Building the Web Application**

In [ ]:
# Here, we have formed a unidirectional link between text_input to generated_text. So whenever the value of the text_input changes, the value of generated_text is changed to the new value as well

text_input.link(generated_text, value='object')
pn.Row(text_input, generated_text)

# creating our button widget

button = pn.widgets.Button(name="Generate",button_type="primary")

# the function of button widget

def click_cb(event):
    pred = get_pred(generated_text.object, model, tok)
    generated_text.object += pred


button.on_click(click_cb)

#adding a title and a brief description

title = pn.pane.Markdown("# **Text Generator**")
desc = pn.pane.HTML("<marquee scrollamount='10'><b> text generator, please enter some starting input text below, click generate a few times and watch the result!</b></marquee>")
app = pn.Column(text_input, button, generated_text);
final_app = pn.Column(title, desc ,app)




In [ ]:
# the Application
final_app.servable()

Column
    [0] Markdown(str)
    [1] HTML(str)
    [2] Column
        [0] TextInput(value='this is a nice', value_input='this is a nice')
        [1] Button(button_type='primary', name='Generate')
        [2] Markdown(str)